# Nuoseklių sakinių klasifikavimas atsitiktinių imčių medicininių tyrimų santraukose

## Modelis 6: Fitting best performing model with all examples of training dataset

In [ ]:
## Bibliotekų importavimas

In [ ]:
# išjungiam TensorFlow įspėjimų (warnings) vaizdavimą rezultatuose
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# glabalios TensorFlow random seed reikšmės nustatymas
tf.random.set_seed(42)

In [ ]:
## Konstantos gautos eksperimentavimo metu

In [ ]:

output_sentence_len = 55

output_sentence_chars_len = 290
max_char_tokens = 28

line_numbers_depth = 17
sentences_total_depth = 22


In [ ]:
## Duomenų paruošimas

In [ ]:
### Universal Sentence Encoder paruošimas modeliui

In [ ]:
# importuojam TensorFlow Hub modulį
import tensorflow_hub as hub

# atsisiunčiam if TF Hub Universal Sentence Encoder
use_embedder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", # nuoroda į iš anksto apmokyta modelį
                                    trainable=False, # nurodome nekeisti modelio išmoktų savybių
                                    name="universal_sentence_encoder")

In [ ]:
# sakinių numerių kodavimas su tf.one_hot
train_line_numbers_onehot = tf.one_hot(train_df["number"].to_numpy(), depth=line_numbers_depth)
test_line_numbers_onehot = tf.one_hot(test_df["number"].to_numpy(), depth=line_numbers_depth)
val_line_numbers_onehot = tf.one_hot(val_df["number"].to_numpy(), depth=line_numbers_depth)

In [ ]:
Dokumente "[**Neural Networks for Joint Sentence Classification in Medical Paper Abstracts**](https://arxiv.org/pdf/1612.05251)" sudarytas ANN modelis pasiekė 89.9% tikslumą po mokymo su visais **PubMed 20k RCT** rinkinio pavyzdžiais. Mūsų geriausias Modelis 5, mokytas su 10% **PubMed 20k RCT** pavyzdžių, santraukų sakinių klasifikaciją atlieka ~85.6% tikslumu.

Šiame etape sudarysim Modelį 6, kuris bus Modelis 5 apmokytas su visais **PubMed 20k RCT** rinkinio pavyzdžiais, mokymą atliekant daugiau nei 5 epochas. Galime susidurti su situacija, kai modelis pradės mokytis iš "*triukšmo*", t.y. po kelis kartus pamatys labai panašius ar net tuos pačius mokymosi pavyzdžius, tai galima pastebėti praradimų (*loss*) kreivių palyginime, kai skirtumas tarp mokymo ir įvertinimo praradimų kreivių staigiai didėja.

Norint išvengti bereikalingo mokymosi iš triukšmo, sudarysime dvi papildomas [TensorFlow Callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks) funkcijas:
- [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) - po kiekvienos epochos išsaugos modelio mokymosi parametrų (*weights*) stadiją jei stebimas parametras pagerėjo.
- [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) - sustabdys tolesnį modelio mokymasi, jei stebimas parametras per nurodytą skaičių mokymosi epochų nepagerėjo.

In [ ]:
# TensorBoard callback funkcijos sudarymas
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir="tensorboard",  # aplankas kur saugoti log failus
)

In [ ]:
# ModelCheckpoint callback funkcijos sudarymas
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoints/m6.ckpt", # failo, kuriame saugosime atskaitos tašką pavadinimas
    monitor="val_accuracy",         # stebimas parametras, kurio atžvilgiu kursime taškus (įvertinimo tikslumas)
    verbose=0,                      # išjungiam funkcijos pranešimus
    save_best_only=True,            # nurodom saugoti tik modelį esant geriausiam stebimam parametrui
    save_weights_only=True,         # saugojam tik įverčius, bet ne visą modelį
    save_freq='epoch',              # kaip dažnai vertinti stebimą parametrą
)

In [ ]:
# EarlyStopping callback fnkcijos sudarymas
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",         # parametras, kurio pokyčiai bus stebimi
    min_delta=0.001,            # dydis nurodantis koks mažiausias stebimo parametro teigiamas pokytis yra priimtinas
    patience=5,                 # kiek epochų laukti stebimo parametro pokyčio, prieš sustabdant mokymą
    verbose=0,                  # išjungiam funkcijos pranešimus
    restore_best_weights=True,  # atstatyti modelio stadiją su geriausiasiais mokymosi rezultatais
)

In [ ]:
# Modelio 5 klonavimas
model_6 = tf.keras.models.clone_model(model_5)
model_6.set_weights(model_5.get_weights())
model_6._name = "model_6"

In [ ]:
# išvedam sudaryto modelio struktūrą
model_6.summary()

In [ ]:
# sukūriam (kompiliuojam) aprašytą modelį
model_6.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# sudarom modelio "Modelis 6" vertinimo diagramas
plot_accuracy_loss_curves(model_6_history, "6")

In [ ]:
# įvertinam modelį su visais testavimo rinkinio pavyzdžiais
# su visais training 20k pavyzdziu apmokytas modelis evaluatina 85.15%
loss, accuracy = model_6.evaluate(test_tokens_chars_pos_dataset)
print(f"Modelio 6 tikslumas su testavimo duomenimis: {(accuracy*100):.2f}%")